# Wood Beam Clips

In [59]:
from functools import partial
import numpy as np
import pandas as pd
from wsweng.wood.dowels import WoodDowel, wood_bolt

## Givens

In [60]:
diameter = [0.75, 0.875, 1.0]  # Nominal diameter, inches
thickness = [3.00, 3.50, 5.125, 5.25, 5.50, 7.0]
clip_thickness = 3/16.0
species = "DFL"
steel = "A36"
make_bolt = partial(
    wood_bolt,
    full_diameter=False,
    main_material=species,
    side_material=steel,
    side_thickness=clip_thickness
)

## Schedule

Calculate Z for each combination of diameter and N

In [64]:
def round_to(x: float, xto: float) -> float:
    return xto*np.ceil(x/xto)

frame_list = []
for dia in diameter:
    edge_distance = round_to(4*dia, 0.25)
    spacing = round_to(4*dia, 0.25)
    for thk in thickness:
        for dbl_v in (False, True):
            z = make_bolt(dia, thk, double_shear=dbl_v).Zv(theta=90.0)
            new_frame = pd.DataFrame(np.arange(2, 6, 1), columns=["N"])
            new_frame["D"] = dia
            new_frame["Thk"] = thk
            new_frame["DblV"] = dbl_v
            new_frame["Height"] = 2*edge_distance + (new_frame["N"] - 1)*spacing
            new_frame["Z"] = z*new_frame["N"]
            frame_list.append(new_frame)

D = pd.concat(frame_list)
D.sample(6)

,N,D,Thk,DblV,Height,Z
1,3,0.875,7.000,False,14.0,2532.155461
0,2,1.000,5.250,False,12.0,1851.455886
1,3,1.000,5.500,True,16.0,6240.705541
1,3,0.875,5.250,False,14.0,2532.155461
2,4,0.875,5.250,False,17.5,3376.207281
2,4,0.875,5.125,False,17.5,3366.231715


## Design Schedule

In [71]:
D_sched = D.copy(deep=True)
D_sched['Z'] = np.round(D["Z"], 1)
D_sched = D_sched.pivot(index=["D", "N", "Height"], columns=["Thk", "DblV"], values="Z")
D_sched

Thk                3.0             3.5  ...      5.5     7.0         
DblV             False    True   False  ...     True   False     True
D     N Height                          ...                          
0.750 2 9.0      946.5  1939.8  1081.6  ...   2578.2  1289.1   2578.2
      3 12.0    1419.8  2909.7  1622.4  ...   3867.2  1933.6   3867.2
      4 15.0    1893.1  3879.6  2163.2  ...   5156.3  2578.2   5156.3
      5 18.0    2366.4  4849.5  2704.0  ...   6445.4  3222.7   6445.4
0.875 2 10.5    1041.4  2116.7  1187.7  ...   3376.2  1688.1   3376.2
      3 14.0    1562.1  3175.0  1781.6  ...   5064.3  2532.2   5064.3
      4 17.5    2082.8  4233.4  2375.4  ...   6752.4  3376.2   6752.4
      5 21.0    2603.5  5291.7  2969.3  ...   8440.5  4220.3   8440.5
1.000 2 12.0    1125.0  2269.3  1280.8  ...   4160.5  2118.6   4237.2
      3 16.0    1687.4  3404.0  1921.2  ...   6240.7  3177.9   6355.8
      4 20.0    2249.9  4538.7  2561.6  ...   8320.9  4237.2   8474.4
      5 24.0    2812.4  5673.4  3201.9  ...  10401.2  5296.5  10593.0

[12 rows x 12 columns]